In [1]:
import tensorflow.keras as keras
from ray.tune import track
import numpy as np
np.random.seed(0)

import tensorflow as tf
print(tf.__version__)
try:
    tf.get_logger().setLevel('INFO')
except Exception as exc:
    print(exc)
import warnings
warnings.simplefilter("ignore")

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint

import ray
from ray import tune

import inspect
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

class TuneReporterCallback(keras.callbacks.Callback):
    """Tune Callback for Keras.
    
    The callback is invoked every epoch.
    """

    def __init__(self, logs={}):
        self.iteration = 0
        super(TuneReporterCallback, self).__init__()

    def on_epoch_end(self, batch, logs={}):
        self.iteration += 1
        track.log(keras_info=logs, mean_accuracy=logs.get("accuracy"), mean_loss=logs.get("loss"))
    '''
    #https://keras.io/guides/writing_your_own_callbacks/ 
    def on_train_begin(self, logs={}):
        self.losses = []
        self.batches = []
        self.weight_masks = model.get_weights()
    
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.batches.append(batch)
        self.model_weights = model.get_weights()
        for i in range(len(self.model_weights)):
            self.model_weights[i] = np.multiply(self.model_weights[i], self.weight_masks[i])
        model.set_weights(self.model_weights)
        self.weight_save.append(model.get_weights)
    '''   
def create_model(learning_rate, dense_1, dense_2):
    assert learning_rate > 0 and dense_1 > 0 and dense_2 > 0, "Did you set the right configuration?"
    model = Sequential()
    model.add(Dense(int(dense_1), input_shape=(98,), activation='relu', name='fc1'))
    model.add(Dense(int(dense_2), activation='relu', name='fc2'))
    model.add(Dense(17, activation='softmax', name='output'))
    optimizer = SGD(lr=learning_rate)
    model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model
        
def tune_AWID(config):  
    model = create_model(learning_rate=config["lr"], dense_1=config["dense_1"], dense_2=config["dense_2"])  # TODO: Change me.
    checkpoint_callback = ModelCheckpoint(
        "model.h5", monitor='loss', save_best_only=True, save_freq=2)

    # Enable Tune to make intermediate decisions by using a Tune Callback hook. This is Keras specific.
    callbacks = [checkpoint_callback, TuneReporterCallback()]
    task_dataset = pickle.load(open('/root/Raytune-Continual-Learning-AWID/task_dataset.pkl', "rb"))
    X_train = task_dataset[0]
    Y_train = task_dataset[1]
    X_test = task_dataset[2]
    Y_test = task_dataset[3]
    # Train the model
    hist = model.fit(
        X_train, Y_train, 
        validation_data=(X_test, Y_test),
        verbose=1, 
        batch_size=100, 
        epochs=5, 
        callbacks=callbacks)
    for key in hist.history:
        print(key)

# Random and uniform sampling for hypertune
def random_search(task_data):
    import numpy as np; np.random.seed(5)  
    hyperparameter_space = {
        "lr": tune.loguniform(0.001, 0.1),  
        "dense_1": tune.uniform(50, 150),
        "dense_2": tune.uniform(20, 100),
    }  
    num_samples = 10  
    ####################################################################################################
    ################ This is just a validation function for tutorial purposes only. ####################
    HP_KEYS = ["lr", "dense_1", "dense_2"]
    assert all(key in hyperparameter_space for key in HP_KEYS), (
        "The hyperparameter space is not fully designated. It must include all of {}".format(HP_KEYS))
    ######################################################################################################

    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False)
    # We clean out the logs before running for a clean visualization later.
    ! rm -rf ~/ray_results/tune_AWID
    analysis = tune.run(
        tune_AWID, 
        verbose=1, 
        config=hyperparameter_space,
        num_samples=num_samples)

    assert len(analysis.trials) > 2, "Did you set the correct number of samples?"

    # Obtain the directory where the best model is saved.
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/val_acc", mode="max")
    print('Best model:',analysis.get_best_trial(metric='keras_info/val_acc', mode='max'), 
          'lr:', analysis.get_best_config(metric='keras_info/val_acc', mode='max')['lr'], 'dense_1:', analysis.get_best_config(metric='keras_info/val_acc', mode='max')['dense_1'], 'dense_2:', analysis.get_best_config(metric='keras_info/val_acc', mode='max')['dense_2']        )
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5")
    tuned_model.summary()
    X_test = task_data[2]
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    return(analysis.get_best_config(metric='keras_info/val_acc', mode='max'))

#PBT population based sampling 
def mutation_pbtsearch(task_data):
    from ray.tune.schedulers import PopulationBasedTraining
    from ray.tune.utils import validate_save_restore
    scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="mean_accuracy",
        mode="max",
        perturbation_interval=5,
        hyperparam_mutations={
            # distribution for resampling
            "lr": lambda: np.random.uniform(0.0001, 1),
            # allow perturbations within this set of categorical values
            "dense_1": [40, 60, 100], "dense_2": [30, 50, 70], 
        }
    )

    old_dirs = os.listdir('/root/ray_results/')

    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False)

    analysis = tune.run(
        tune_AWID,
        name="PBT_AWID",
        scheduler=scheduler,
        reuse_actors=True,
        verbose=1,
        stop={
            "training_iteration": 20,
        },
        num_samples=10,

        # PBT starts by training many neural networks in parallel with random hyperparameters. 
        config={
            "lr": tune.uniform(0.001, 1),
            "dense_1": tune.uniform(50, 150), "dense_2": tune.uniform(20, 100),
        })
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/val_acc", mode="max")
    print('Best model:',analysis.get_best_trial(metric='keras_info/val_acc', mode='max'), 
          analysis.get_best_config(metric='keras_info/val_acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5")
    tuned_model.summary()
    X_test = task_data[2]
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    return(analysis.get_best_config(metric='keras_info/val_acc', mode='max'))

#ASHA Schedular
def ASHA_search(task_data):
    from ray.tune.schedulers import ASHAScheduler
    ray.shutdown()  # Restart Ray defensively in case the ray connection is lost. 
    ray.init(log_to_driver=False)
    custom_scheduler = ASHAScheduler(
        metric='episode_reward_mean',
        mode="max",
        reduction_factor = 2,
        grace_period=1)# TODO: Add a ASHA as custom scheduler here
    hyperparameter_space={
        "lr": tune.uniform(0.001, 1),
            "dense_1": tune.uniform(50, 150), "dense_2": tune.uniform(20, 100),
        }
    
    analysis = tune.run(
        tune_AWID, 
        scheduler=custom_scheduler, 
        config=hyperparameter_space, 
        verbose=1,
        num_samples=10,
        #resources_per_trial={"cpu":4},
        name="ASHA_AWID"  # This is used to specify the logging directory.
    )
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/val_acc", mode="max")
    print('Best model:',analysis.get_best_trial(metric='keras_info/val_acc', mode='max'), 
          analysis.get_best_config(metric='keras_info/val_acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5")
    tuned_model.summary()
    X_test = task_data[2]
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    return(analysis.get_best_config(metric='keras_info/val_acc', mode='max'))


#HyperOpt Search 
def hyperopt_search(task_data):
    from hyperopt import hp
    from ray.tune.suggest.hyperopt import HyperOptSearch

    # This is a HyperOpt specific hyperparameter space configuration.
    space = {
            "lr": hp.loguniform("lr", -10, -1),
            "dense_1": hp.loguniform("dense_1", 0.3, 3), "dense_2": hp.loguniform("dense_2", 0.2, 2),
        }
    # Create a HyperOptSearch object by passing in a HyperOpt specific search space.
    # Also enforce that only 1 trials can run concurrently.
    hyperopt_search = HyperOptSearch(space, max_concurrent=1, metric="mean_loss", mode="min") 

    # We Remove the dir so that we can visualize tensorboard correctly
    ! rm -rf ~/ray_results/search_algorithm 
    
    analysis = tune.run(
        tune_AWID, 
        num_samples=10,  
        search_alg=hyperopt_search,
        verbose=1,
        name="AWID_search_algorithm",
    )
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/val_acc", mode="max")
    print('Best model:',analysis.get_best_trial(metric='keras_info/val_acc', mode='max'), 
          analysis.get_best_config(metric='keras_info/val_acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5")
    tuned_model.summary()
    X_test = task_data[2]
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    return(analysis.get_best_config(metric='keras_info/val_acc', mode='max'))

#Bayesian Search
#https://docs.ray.io/en/latest/tune-searchalg.html#bohb
#https://github.com/ray-project/ray/blob/master/python/ray/tune/examples/bohb_example.py
#https://github.com/ray-project/ray/blob/master/python/ray/tune/suggest/bohb.py
def Bayesian_search(task_data):
    from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
    from ray.tune.suggest.bohb import TuneBOHB
    import ConfigSpace as CS
    config_space = CS.ConfigurationSpace()
    config_space.add_hyperparameter(
        CS.UniformFloatHyperparameter("lr", lower=0.001, upper=0.1))
    config_space.add_hyperparameter(
        CS.UniformFloatHyperparameter("dense_1", lower=50, upper=150))
    config_space.add_hyperparameter(
        CS.UniformFloatHyperparameter("dense_2", lower=20, upper=100))
    
    bohb_hyperband = HyperBandForBOHB(
        time_attr="training_iteration",
        max_t=3,
        reduction_factor=4,
        metric='mean_accuracy', 
        mode='min')
    bohb_search = TuneBOHB(
        config_space, max_concurrent=2, metric='mean_loss', 
        mode='min')
    
    tune.run(tune_AWID,
        name="Bayseian_AWID",
        scheduler=bohb_hyperband,
        search_alg=bohb_search,
        num_samples=10,
        stop={"training_iteration": 50})
    from ray.tune import Analysis as analysis
    analysis = ray.tune.Analysis('/root/ray_results/Bayseian_AWID') 
    print("You can use any of the following columns to get the best model: \n{}.".format(
        [k for k in analysis.dataframe() if k.startswith("keras_info")]))
    print("=" * 10)
    logdir = analysis.get_best_logdir("keras_info/val_acc", mode="max")
    print('Best model:', analysis.get_best_config(metric='keras_info/val_acc', mode='max'))
    # We saved the model as `model.h5` in the logdir of the trial.
    from tensorflow.keras.models import load_model
    tuned_model = load_model(logdir + "/model.h5")
    tuned_model.summary()
    X_test = task_data[2]
    Y_test = task_data[3]
    tuned_loss, tuned_accuracy = tuned_model.evaluate(X_test, Y_test, verbose=0)
    print("Loss is {:0.4f}".format(tuned_loss))
    print("Tuned accuracy is {:0.4f}".format(tuned_accuracy))
    return(analysis.get_best_config(metric='keras_info/val_acc', mode='max'))

1.14.0


In [3]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 12 19:27:34 2018

@author: Jason
"""
import warnings
warnings.simplefilter("ignore")
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import numpy as np
from evaluate import evaluate
from policy_gradient import Controller
import argparse
import datetime
import time
import pickle
import pdb

def dump_best_params(params, task_id):
        f = open('param_task'+str(task_id)+'.pkl', "wb")
        pickle.dump(params, f)
        f.close()

class RCL:
    def __init__(self,args):
        self.args = args
        self.num_tasks = args.n_tasks
        self.epochs = args.n_epochs
        self.batch_size = args.batch_size
        self.lr = args.lr
        self.data_path = args.data_path
        self.max_trials = args.max_trials
        self.penalty = args.penalty
        self.task_list = self.create_mnist_task()
        self.evaluates = evaluate(task_list=self.task_list, args = args)
        self.train()

    def create_session(self):
        config = tf.ConfigProto(allow_soft_placement=True)
        config.gpu_options.allow_growth = True
        sess = tf.Session(config=config)
        return sess
    
    def create_mnist_task(self):
        data = pickle.load(open(self.data_path, "rb"))
        return data

    def train(self):
        self.best_params={}
        self.result_process = []
        for task_id in range(0,self.num_tasks):
            self.best_params[task_id] = [0,0]
            if task_id == 0:
                with tf.Graph().as_default() as g:
                    with tf.name_scope("before"):
                        #call one of the search algorithm
                        !rm -rf task_dataset.pkl
                        f = open('task_dataset.pkl', 'wb')
                        pickle.dump(self.task_list[task_id], f)
                        f.close()
                        hyper_param = Bayesian_search(self.task_list[task_id])
                        inputs = tf.placeholder(shape=(None, 98), dtype=tf.float32)
                        y = tf.placeholder(shape=(None, 17), dtype=tf.float32)
                        w1 = tf.Variable(tf.truncated_normal(shape=(98,int(hyper_param['dense_1'])), stddev=0.01))
                        b1 = tf.Variable(tf.constant(0.1, shape=(int(hyper_param['dense_1']),)))
                        w2 = tf.Variable(tf.truncated_normal(shape=(int(hyper_param['dense_1']),int(hyper_param['dense_2'])), stddev=0.01))
                        b2 = tf.Variable(tf.constant(0.1, shape=(int(hyper_param['dense_2']),)))
                        w3 = tf.Variable(tf.truncated_normal(shape=(int(hyper_param['dense_2']),17), stddev=0.01))
                        b3 = tf.Variable(tf.constant(0.1, shape=(17,)))
                        output1 = tf.nn.relu(tf.nn.xw_plus_b(inputs,w1,b1,name="output1"))
                        output2 = tf.nn.relu(tf.nn.xw_plus_b(output1,w2,b2,name="output2"))
                        output3 = tf.nn.xw_plus_b(output2,w3,b3,name="output3")
                        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output3)) + \
                               0.0001*(tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3))
                        if self.args.optimizer=="adam":
                            optimizer = tf.train.AdamOptimizer(learning_rate=self.args.lr)
                        elif self.args.optimizer=="rmsprop":
                            optimizer = tf.train.RMSPropOptimizer(learning_rate=self.lr)
                        elif self.args.optimizer=="sgd":
                            optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.lr)
                        else:
                            raise Exception("please choose one optimizer")
                        train_step = optimizer.minimize(loss)
                        accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y,axis=1),tf.argmax(output3,axis=1)),tf.float32))
                        sess = self.create_session()
                        sess.run(tf.global_variables_initializer())
                        l = len(self.task_list[0][1])
                        for epoch in range(self.epochs):
                            flag = 0
                            for _ in range(l//self.batch_size+1):
                                batch_xs, batch_ys = (self.task_list[task_id][0][flag:flag+self.batch_size],self.task_list[task_id][1][flag:flag+self.batch_size])
                                flag += self.batch_size
                                sess.run(train_step,feed_dict={inputs:batch_xs, y:batch_ys})
                        # 4, 5 are the indicies of testing dataset of task tuple. 1,2, are the indices of training and 3,4 are the indices of validation. 
                        accuracy_test = sess.run(accuracy, feed_dict={inputs:self.task_list[task_id][4], y:self.task_list[task_id][5]})
                        print("Task {}. Test accuracy: {} ".format(task_id,accuracy_test))
                        self.vars = sess.run([w1,b1,w2,b2,w3,b3])
                    self.best_params[task_id] = [accuracy_test,self.vars]
            else:
                tf.reset_default_graph()
                controller = Controller(self.args)
                results = []
                best_reward = 0
                for trial in range(self.max_trials):
                    actions = controller.get_actions()
                    #call one of the search algorithm
                    !rm -rf task_dataset.pkl
                    f = open('task_dataset.pkl', 'wb')
                    pickle.dump(self.task_list[task_id], f)
                    f.close()
                    hyper_param = Bayesian_search(self.task_list[task_id])
                    actions = [int(hyper_param['dense_1']), int(hyper_param['dense_2'])]
                    print("***actions***: ",actions)
                    accuracy_val, accuracy_test = self.evaluates.evaluate_action(var_list = self.vars, 
                             actions=actions, task_id = task_id)

                    results.append(accuracy_val)
                    print("Task {}, Trial {}, Test accuracy: {} ".format(task_id, trial, accuracy_test))
                    reward = accuracy_val - self.penalty*sum(actions)
                    print("reward: ", reward)
                    if reward > best_reward:
                        best_reward = reward
                        self.best_params[task_id] = (accuracy_test, self.evaluates.var_list)
                    controller.train_controller(reward)
                print('Best architecture {}. Reward {}. Test accuracy {}'.format([arch.shape for arch in self.best_params[task_id][1]], reward, self.best_params[task_id][0]))
                print('-' * 150)
                controller.close_session()
                self.result_process.append(results)
                self.vars = self.best_params[task_id][1]
            dump_best_params(self.vars, task_id)
        
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Reinforced Continual learning')

    # model parameters
    parser.add_argument('--n_tasks', type=int, default=5,
                        help='number of tasks')
    parser.add_argument('--n_hiddens', type=str, default='60,30',
                        help='number of hidden neurons at each layer')
    parser.add_argument('--n_layers', type=int, default=2,
                        help='number of hidden layers')

    # optimizer parameters
    parser.add_argument('--n_epochs', type=int, default=10,
                        help='Number of epochs per task')
    parser.add_argument('--batch_size', type=int, default=64,
                        help='batch size')
    parser.add_argument('--lr', type=float, default=1e-3,
                        help='SGD learning rate')
    parser.add_argument('--max_trials', type=int, default=1,
                        help='max_trials')

    # experiment parameters
    parser.add_argument('--seed', type=int, default=0,
                        help='random seed')

    parser.add_argument('--save_path', type=str, default='./results/',
                        help='save models at the end of training')

    # data parameters
    parser.add_argument('--data_path', default='./data/awid_tasks.pkl',
                        help='path where data is located')
    parser.add_argument('--state_space', type=int, default=30, help="the state space for search") 
    parser.add_argument('--actions_num', type=int, default=2, help="how many actions to dscide")
    parser.add_argument('--hidden_size', type=int, default=100, help="the hidden size of RNN")
    parser.add_argument('--num_layers', type=int, default=2, help="the layer of a RNN cell")
    parser.add_argument('--cuda', type=bool, default=True, help="use GPU or not")
    parser.add_argument('--bendmark', type=str, default='critic', help="the type of bendmark")
    parser.add_argument('--penalty', type=float, default=0.0001, help="the type of bendmark")#0.0001
    parser.add_argument('--optimizer', type=str, default="adam", help="the type of optimizer")#
    parser.add_argument('--method', type=str, default='Bayesian_search', help="method for generate actions")

    args = parser.parse_args("")
    start = time.time()
    jason = RCL(args)  
    end = time.time()
    params = jason.best_params
    if not os.path.exists(args.save_path):
        os.makedirs(args.save_path)
    fname = "RCL_FC_" + args.data_path.split('/')[-1] + "_" + datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
    fname += '_' + str(args.lr) + str("_") + str(args.n_epochs) + '_' + str(args.max_trials) + '_' + str(args.batch_size) + \
             '_' + args.bendmark + '_' + str(args.penalty) + '_' + args.optimizer + '_' + str(args.state_space) + '_' + \
             str(end-start) + '_' + args.method
    fname = os.path.join(args.save_path, fname)
    f = open(fname + '.txt', 'w')
    accuracy = []
    for index,value in params.items():
        print([_.shape for _ in value[1]], file=f)
        accuracy.append(value[0])
    print(accuracy,file=f)
    f.close()
    print(fname)
    name = fname + '.pkl'
    f = open(name, 'wb')
    pickle.dump(jason.result_process, f)
    f.close()


Trial name,status,loc,dense_1,dense_2,lr
tune_AWID_f8c62502,RUNNING,,52.3023,38.7508,0.0706179
tune_AWID_f8c62503,PENDING,,59.6073,27.7255,0.0485014


Result for tune_AWID_f8c62502:
  date: 2020-10-26_07-22-17
  done: false
  experiment_id: 35e55291e9a948df8e0c7d150a37aeb3
  experiment_tag: 1_dense_1=52.302,dense_2=38.751,lr=0.070618
  hostname: ip-172-31-19-28
  iterations_since_restore: 1
  keras_info:
    acc: 0.46701696515083313
    loss: 1.039676358527456
    val_acc: 0.48701488971710205
    val_loss: 0.9006623275467144
  mean_loss: 1.039676358527456
  neg_mean_loss: -1.039676358527456
  node_ip: 172.31.19.28
  pid: 17258
  time_since_restore: 0.8659436702728271
  time_this_iter_s: 0.8659436702728271
  time_total_s: 0.8659436702728271
  timestamp: 1603696937
  timesteps_since_restore: 0
  training_iteration: 0
  trial_id: f8c62502
  
Result for tune_AWID_f8c62503:
  date: 2020-10-26_07-22-17
  done: false
  experiment_id: 8139d6fca0754651899abf84d3ce10fd
  experiment_tag: 2_dense_1=59.607,dense_2=27.726,lr=0.048501
  hostname: ip-172-31-19-28
  iterations_since_restore: 1
  keras_info:
    acc: 0.5012426376342773
    loss: 0.962

Trial name,status,loc,dense_1,dense_2,lr,loss,iter,total time (s)
tune_AWID_f8c62502,TERMINATED,,52.3023,38.7508,0.0706179,0.856073,3,1.76303
tune_AWID_f8c62503,TERMINATED,,59.6073,27.7255,0.0485014,0.761421,3,1.92536
tune_AWID_fa27dd92,RUNNING,172.31.19.28:17242,116.86,31.2769,0.0645389,1.07448,0,0.934326
tune_AWID_fa27dd95,RUNNING,,89.3381,49.0994,0.0383714,,,


Result for tune_AWID_fa27dd95:
  date: 2020-10-26_07-22-21
  done: false
  experiment_id: 70d5ef5969d5447d97c9a07ecd5f4958
  experiment_tag: 4_dense_1=89.338,dense_2=49.099,lr=0.038371
  hostname: ip-172-31-19-28
  iterations_since_restore: 1
  keras_info:
    acc: 0.4920826554298401
    loss: 1.040037864507163
    val_acc: 0.5617157816886902
    val_loss: 0.7924696815963524
  mean_loss: 1.040037864507163
  neg_mean_loss: -1.040037864507163
  node_ip: 172.31.19.28
  pid: 17379
  time_since_restore: 0.9458680152893066
  time_this_iter_s: 0.9458680152893066
  time_total_s: 0.9458680152893066
  timestamp: 1603696941
  timesteps_since_restore: 0
  training_iteration: 0
  trial_id: fa27dd95
  
Result for tune_AWID_fa27dd95:
  date: 2020-10-26_07-22-22
  done: true
  experiment_id: 70d5ef5969d5447d97c9a07ecd5f4958
  experiment_tag: 4_dense_1=89.338,dense_2=49.099,lr=0.038371
  hostname: ip-172-31-19-28
  iterations_since_restore: 4
  keras_info:
    acc: 0.6722289323806763
    loss: 0.764929

2020-10-26 07:22:23,040	ERROR worker.py:1030 -- Possible unhandled error from worker: ray::WrappedTrackFunc.save_to_object() (pid=17258, ip=172.31.19.28)
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 417, in ray._raylet.execute_task.function_executor
  File "/usr/local/lib/python3.6/dist-packages/ray/tune/trainable.py", line 385, in save_to_object
    checkpoint_path = self.save(tmpdir)
  File "/usr/local/lib/python3.6/dist-packages/ray/tune/trainable.py", line 342, in save
    checkpoint = self._save(checkpoint_dir)
  File "/usr/local/lib/python3.6/dist-packages/ray/tune/trainable.py", line 622, in _save
    raise NotImplementedError
NotImplementedError


...

Trial name,status,loc,dense_1,dense_2,lr,loss,iter,total time (s)
tune_AWID_314140ce,TERMINATED,,54.3869,88.8734,0.042811,0.720242,3,0.8492
tune_AWID_314140cf,TERMINATED,,80.9835,86.3313,0.0249163,0.668352,3,0.914692
tune_AWID_32c144e0,TERMINATED,,100.476,60.5525,0.0492941,0.749423,3,0.845386
tune_AWID_32c144e3,TERMINATED,,118.154,21.2917,0.0738506,0.769073,3,0.885932
tune_AWID_3422fc27,TERMINATED,,140.748,57.8638,0.0652532,0.792964,3,0.827754
tune_AWID_3422fc2d,TERMINATED,,61.3735,27.8502,0.0840462,0.785153,3,0.859437
tune_AWID_35cc1491,TERMINATED,,83.7668,30.3101,0.0177144,0.715829,3,0.830426
tune_AWID_35cc1497,TERMINATED,,111.842,37.0387,0.0474753,0.725804,3,0.869158
tune_AWID_377b7eb7,TERMINATED,,123.456,63.7203,0.0255676,0.721518,3,0.90122
tune_AWID_377b7ebd,TERMINATED,,52.0922,92.9835,0.00248269,0.753103,3,0.926085


You can use any of the following columns to get the best model: 
['keras_info/loss', 'keras_info/acc', 'keras_info/val_loss', 'keras_info/val_acc'].
Best model: {'dense_1': 78.76604919303485, 'dense_2': 33.94799068429313, 'lr': 0.02052911406111622}
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 78)                7722      
_________________________________________________________________
fc2 (Dense)                  (None, 33)                2607      
_________________________________________________________________
output (Dense)               (None, 17)                578       
Total params: 10,907
Trainable params: 10,907
Non-trainable params: 0
_________________________________________________________________
Loss is 7.4970
Tuned accuracy is 0.0000
***actions***:  [78, 33]


2020-10-26 07:24:05,381	ERROR worker.py:1030 -- Possible unhandled error from worker: ray::WrappedTrackFunc.save_to_object() (pid=18612, ip=172.31.19.28)
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 417, in ray._raylet.execute_task.function_executor
  File "/usr/local/lib/python3.6/dist-packages/ray/tune/trainable.py", line 385, in save_to_object
    checkpoint_path = self.save(tmpdir)
  File "/usr/local/lib/python3.6/dist-packages/ray/tune/trainable.py", line 342, in save
    checkpoint = self._save(checkpoint_dir)
  File "/usr/local/lib/python3.6/dist-packages/ray/tune/trainable.py", line 622, in _save
    raise NotImplementedError
NotImplementedError


task:4, epoch:0, test accuracy:0.9447853
task:4, epoch:4, test accuracy:1.0
task:4, epoch:8, test accuracy:1.0
task:4, epoch:9, test accuracy:1.0
Task 4, Trial 0, Test accuracy: 1.0 
reward:  0.9889
Best architecture [(98, 321), (321,), (321, 252), (252,), (252, 17), (17,)]. Reward 0.9889. Test accuracy 1.0
------------------------------------------------------------------------------------------------------------------------------------------------------
./results/RCL_FC_awid_tasks.pkl_2020-10-26_07:24:07_0.001_10_1_64_critic_0.0001_adam_30_113.96204972267151_Bayesian_search


2020-10-26 07:24:08,383	ERROR worker.py:1030 -- Possible unhandled error from worker: ray::WrappedTrackFunc.save_to_object() (pid=18670, ip=172.31.19.28)
  File "python/ray/_raylet.pyx", line 463, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 417, in ray._raylet.execute_task.function_executor
  File "/usr/local/lib/python3.6/dist-packages/ray/tune/trainable.py", line 385, in save_to_object
    checkpoint_path = self.save(tmpdir)
  File "/usr/local/lib/python3.6/dist-packages/ray/tune/trainable.py", line 342, in save
    checkpoint = self._save(checkpoint_dir)
  File "/usr/local/lib/python3.6/dist-packages/ray/tune/trainable.py", line 622, in _save
    raise NotImplementedError
NotImplementedError
